# Bianco's CNN color constancy model recreated

#### CS 7180 - Advanced Perception
#### Di Zhang & Prakriti Pritmani
#### Oct 14, 2023

## imported packages

In [13]:
import numpy as np
import cv2
import scipy.io
import glob
from random import randint
import progressbar as pb
import h5py
import numpy as np
import tensorflow as tf
from tensorflow import keras

from keras import layers, optimizers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.initializers import glorot_uniform
from keras import backend as K
from keras.preprocessing import image



## Helper Functions

In [4]:
#define progress timer class
class progress_timer:

    def __init__(self, n_iter, description="Something"):
        self.n_iter         = n_iter
        self.iter           = 0
        self.description    = description + ': '
        self.timer          = None
        self.initialize()

    def initialize(self):
        #initialize timer
        widgets = [self.description, pb.Percentage(), ' ',   
                   pb.Bar('=', '[', ']'), ' ', pb.ETA()]
        self.timer = pb.ProgressBar(widgets=widgets, maxval=self.n_iter).start()

    def update(self, q=1):
        #update timer
        self.timer.update(self.iter)
        self.iter += q

    def finish(self):
        #end timer
        self.timer.finish()
        

## Data Preprocesssing

In [5]:
# path_mat = '/Users/dizhang/Desktop/NEU/CS7180_Advanced_Perception/assignment/CS7180_Bianco_CNN_CC_model/NikonD40_gt.mat'
# path = '/Users/dizhang/Desktop/NEU/CS7180_Advanced_Perception/assignment/CS7180_Bianco_CNN_CC_model/RAW/'
# gt_mat = scipy.io.loadmat(path_mat, squeeze_me = True, struct_as_record = False)
# gt_illum = gt_mat['groundtruth_illuminants'][:5][1]
# print(gt_illum)

# flist = glob.glob(path + '*.NEF')

# print(flist)

# image_number = flist[0];
# index = (image_number.replace(path ,'')).replace('.NEF', '').replace('NikonD40_', '');


# print(index)

In [6]:
def generate_data(train_size, patch_size):

    mat_path = '/Users/dizhang/Desktop/NEU/CS7180_Advanced_Perception/assignment/CS7180_Bianco_CNN_CC_model/NikonD40_gt.mat'
    path = '/Users/dizhang/Desktop/NEU/CS7180_Advanced_Perception/assignment/CS7180_Bianco_CNN_CC_model/RAW_train/'

    illum_mat = scipy.io.loadmat(mat_path, squeeze_me = True, struct_as_record = False)
    ground_truth_illum = illum_mat['groundtruth_illuminants']
    
    flist = glob.glob(path + '*.NEF')
    number_of_gt = len(flist)
    
    pt = progress_timer(n_iter = number_of_gt, description = 'Generating Training Data :')
    
    patches_per_image = int(train_size/number_of_gt)

    X_origin, Y_origin, name_train = [], [], []
    i = 0
    patch_r, patch_c = patch_size

    while (i < number_of_gt):
        
        image_number = flist[i]
        index = (image_number.replace(path ,'')).replace('.NEF', '').replace('NikonD40_', '')
        
        image = cv2.imread(image_number)
        n_r, n_c, _ = np.shape(image)
        total_patch = int(((n_r - n_r%patch_r)/patch_r)*((n_c - n_c%patch_c)/patch_c))
        
        img_resize = cv2.resize(image, ((n_r - n_r%patch_r), (n_c - n_c%patch_c)))
        img_reshape = np.reshape(img_resize, (int(patch_r), -1, 3))
        
        #Create CLAHE object
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    
        for j in range (0, patches_per_image):
            
            rd = randint(0, total_patch - 1)
            img_patch = img_reshape[0:patch_r, rd*patch_c:(rd+1)*patch_c]
            
            #Convert image to Lab to perform contrast normalizing
            lab= cv2.cvtColor(img_patch, cv2.COLOR_BGR2LAB)
            
            #Contrast normalizing(Stretching)
            l, a, b = cv2.split(lab)
            cl = clahe.apply(l)
            clab = cv2.merge((cl, a, b))
            
            #Convert back to BGR
            img_patch = cv2.cvtColor(clab, cv2.COLOR_LAB2BGR)
            
            img_patch = cv2.cvtColor(img_patch, cv2.COLOR_BGR2RGB)
            
            X_origin.append(img_patch)
            Y_origin.append(ground_truth_illum[int(index) - 1])
        
        name_train.append('%04d' % (int(index) - 1))
             
        i += 1
        
        pt.update()
 
    X_origin = np.asarray(X_origin)
    Y_origin = np.asarray(Y_origin)
    
    X_origin = X_origin/255
    max_Y = np.amax(Y_origin, 1)
    Y_origin[:, 0] = Y_origin[:, 0]/max_Y
    Y_origin[:, 1] = Y_origin[:, 1]/max_Y
    Y_origin[:, 2] = Y_origin[:, 2]/max_Y
    
    seed = randint(1, 5000)
    np.random.seed(seed)
    X_origin = np.random.permutation(X_origin)
    
    np.random.seed(seed)
    Y_origin = np.random.permutation(Y_origin)
    
    pt.finish()
    
    return X_origin, Y_origin, name_train

In [7]:
train_size = 1600
patch_size = (32, 32)

X_train, Y_train, name_train = generate_data(train_size, patch_size)
np.save('X_train.npy', X_train)
np.save('Y_train.npy', Y_train) 
np.save('name_train.npy', name_train)

Generating Training Data :: 100% [==============================] Time: 0:00:00


In [8]:
test_size = 740
patch_size = (32, 32)

X_test, Y_test, name_test = generate_data(test_size, patch_size)
np.save('X_test.npy', X_test)
np.save('Y_test.npy', Y_test) 
np.save('Y_test.npy', name_test)

Generating Training Data :: 100% [==============================] Time: 0:00:00


In [9]:
print(len(X_test))

720


## Model

In [10]:
def ColorNet(input_shape, channels = 3):
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Stage 1
    X = Conv2D(240, (1, 1), strides = (1, 1), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X_input)
    X = BatchNormalization(name = 'bn_conv1')(X)
    
    X = MaxPooling2D((8, 8), strides=(8, 8))(X)
  
    # output layer
    X = Flatten()(X)
    X = Dense(40, activation='relu', name='fc' + str(40))(X);
    X = Dropout(rate = 0.5)(X);
    
    X = Dense(channels, activation=None, name='fc' + str(channels))(X);
    
    # Create model
    color_model = Model(inputs = X_input, outputs = X, name='ColorNet');
    
    return color_model;

## Training

In [15]:
loss_cos_sim = tf.keras.losses.CosineSimilarity()
rmsprop = optimizers.legacy.RMSprop(lr = 0.001, rho=0.9, epsilon=None, decay=0.0);

cc_model = ColorNet(input_shape = X_train.shape[1:4]);
cc_model.compile(optimizer = 'Adam', loss = loss_cos_sim , metrics = ['accuracy']);

estimate = cc_model.fit(X_train, Y_train, validation_split = 0.3333, epochs = 20, batch_size = 160);

preds = cc_model.evaluate(X_test, Y_test);
print();
print ("Loss = " + str(preds[0]));
print ("Test Accuracy = " + str(preds[1]));

# serialize model to JSON
model_json = cc_model.to_json()
with open("cc_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cc_model.save_weights("cc_model.h5")
print("Saved model to disk")
 

Epoch 1/20
7/7 [==============================] - 1s 117ms/step - loss: -0.6129 - accuracy: 0.4841 - val_loss: -0.9817 - val_accuracy: 0.9813
Epoch 2/20
7/7 [==============================] - 1s 100ms/step - loss: -0.9030 - accuracy: 0.6689 - val_loss: -0.9831 - val_accuracy: 0.9813
Epoch 3/20
7/7 [==============================] - 1s 100ms/step - loss: -0.9108 - accuracy: 0.7486 - val_loss: -0.9801 - val_accuracy: 0.9813
Epoch 4/20
7/7 [==============================] - 1s 100ms/step - loss: -0.9246 - accuracy: 0.7777 - val_loss: -0.9809 - val_accuracy: 0.9813
Epoch 5/20
7/7 [==============================] - 1s 99ms/step - loss: -0.9339 - accuracy: 0.7758 - val_loss: -0.9839 - val_accuracy: 0.9813
Epoch 6/20
7/7 [==============================] - 1s 107ms/step - loss: -0.9365 - accuracy: 0.7992 - val_loss: -0.9860 - val_accuracy: 0.9813
Epoch 7/20
7/7 [==============================] - 1s 104ms/step - loss: -0.9412 - accuracy: 0.7917 - val_loss: -0.9865 - val_accuracy: 0.9813
Epoch 8

## 

## Result